# 机器学习策略（1）


## 1. 为什么需要机器学习策略

改进机器学习模型，可以做的事情有很多，效果各不相同。如何去分析机器学习问题，让模型更快地达到理想的状态，这是机器学习策略的意义。

![Motivating example](img/Motivating example.png)

## 2. 正交化

正交化是一种系统设计的特性，它可以保证算法的一个指令或组件，不会对系统的其它组件产生副作用。这个特性使得验证算法的工具各自独立，从而减轻测试和开发周期。

在设计监督学习系统时，下面是4个假设，我们需要这些假设都成立，并且相互正交：

1. 成本函数在训练集上表现良好
    - 如果不好，使用更大的神经网络，或者使用更好的优化算法
2. 成本函数在开发集上表现良好
    - 如果不好，使用正则化，或者使用更大的训练集
3. 成本函数在测试集上表现良好
    - 如果不好，使用更大的开发集
4. 模型在现实世界里表现良好
    - 如果不好，开发集可能设置错误，或者所选的成本函数不是衡量所给问题好坏的一个很好指标

![Chain of assumptions in ML](img/Chain of assumptions in ML.png)